## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 
from sklearn.svm import LinearSVC

/home/lab_services_student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-09-26 11:42:46.154181: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-26 11:42:46.156494: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 11:42:46.163301: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 11:42:46.182395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting

## Data Transforming

In [2]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [3]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,0,1,225,308751,0
798459,0,6,0,0,1,0,0,0,3,0,0,0,0,1,690,308750,0
798460,0,9,0,0,2,0,0,0,3,0,0,0,0,1,215,308771,0
798461,0,16,0,0,2,0,0,0,10,0,0,0,0,1,219,308776,0


In [4]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [5]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [6]:
c=data.subdomain_encoded.nunique()
data.subdomain_encoded.nunique()

111203

In [7]:
d=a+b+c
d

421218

In [8]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29


In [9]:

y=data['status']
X=data.drop('status', axis=1)
col=X.columns
X=StandardScaler().fit_transform(X)
X=pd.DataFrame(X,columns=col)


In [10]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=54)

In [11]:
X_train.shape

(558924, 16)

In [12]:
X_test.shape

(239539, 16)

In [13]:
X_train

,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
565278,-0.527616,-0.544389,-0.363395,-0.750244,-0.806292,-0.143633,-0.065063,-0.760534,-0.211449,-0.489272,-0.116224,-0.424454,-0.218199,-0.551873,0.310505,-0.591072
563177,-0.527616,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,-0.966244,-0.211449,-0.489272,-0.116224,-0.424454,0.343734,2.562604,0.283721,-0.591072
680861,0.312164,1.014999,-0.175586,-0.072874,0.826390,-0.143633,-0.065063,1.502273,1.682923,-0.489272,-0.116224,-0.424454,-0.780133,-0.551873,-1.641480,-0.591072
707757,-0.191704,-0.544389,-0.363395,-0.750244,1.914844,-0.143633,-0.065063,-0.349115,-0.211449,-0.489272,-0.116224,-0.424454,0.343734,-0.551873,-0.145213,-0.591072
372716,-0.143717,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,0.062305,-0.211449,-0.489272,-0.116224,-0.424454,0.343734,1.672753,-0.125565,-0.591072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224993,-0.575604,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,-0.489272,-0.116224,-0.424454,-0.218199,-0.551873,-0.733869,-0.591072
267287,-0.239692,-0.544389,-0.363395,0.604495,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,0.497542,-0.116224,-0.424454,-0.218199,-0.842915,0.152235,0.867637
323343,-0.143717,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,0.497542,-0.116224,-0.424454,0.343734,-0.551873,0.028666,0.021798
367941,0.456126,-0.154542,-0.175586,-0.072874,-0.262065,-0.143633,-0.065063,-0.966244,-0.211449,0.497542,-0.116224,-0.424454,0.343734,-0.551873,1.476044,0.884194


## Traditional Models

In [14]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
# trains in around 2 minutes and has 0.95 accross the board

              precision    recall  f1-score   support

           0       0.95      0.94      0.95    111486
           1       0.95      0.96      0.95    128053

    accuracy                           0.95    239539
   macro avg       0.95      0.95      0.95    239539
weighted avg       0.95      0.95      0.95    239539



In [15]:
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
y1=df['status']
X1=tfidf_df
X_train1, X_test1, y_train1,y_test1 = train_test_split(X,y,test_size=0.2,random_state=504)


In [16]:
# rf = RandomForestClassifier()
# rf.fit(X_train1, y_train1)
# y_pred1 = rf.predict(X_test1)
# print(classification_report(y_test1, y_pred1))
# took 32 minutes to train on unstructured data, was less accurate for the most part 

In [17]:
clf = svm.SVC(kernel='linear',C=1.0) 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
# ran for 5 hours and did not finish training